In [20]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_openai import ChatOpenAI
from llama_index.llms.langchain import LangChainLLM
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex
from llama_index.core.schema import Document
from langchain.schema import SystemMessage, HumanMessage
import openai
import os

import json
with open("../keys/keys.json", "r") as fi:
    api_key = json.load(fi)['api_key']

os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

In [2]:
import os
os.getcwd()

'c:\\Users\\lexil\\Documents\\NSS_Projects\\nlp-06-rag-Jorgen85Lex\\notebooks'

In [3]:
with open("../data/2505.23724v1.txt", "r", encoding="utf-8") as f:
    text = f.read()

print(text[:1000])

arXiv:2505.23724v1  [cs.LG]  29 May 2025SC-LoRA: Balancing Efficient Fine-tuning and Knowledge Preservation via
Subspace-Constrained LoRA
Minrui Luo∗1,2, Fuhang Kuang∗1, Yu Wang3, Zirui Liu1, Tianxing He†1,2
1Institute for Interdisciplinary Information Sciences, Tsinghua University
2Shanghai Qi Zhi Institute
3Institute of Information Engineering, Chinese Academy of Sciences
{luomr22,kfh22,liu-zr22}@mails.tsinghua.edu.cn;
wangyu2002@iie.ac.cn hetianxing@mail.tsinghua.edu.cn
Abstract
Parameter-Efficient Fine-Tuning (PEFT)
methods, particularly Low-Rank Adaptation
(LoRA), are indispensable for efficiently
customizing Large Language Models (LLMs).
However, vanilla LoRA suffers from slow
convergence speed and knowledge forgetting
problems. Recent studies have leveraged
the power of designed LoRA initialization,
to enhance the fine-tuning efficiency, or to
preserve knowledge in the pre-trained LLM.
However, none of these works can address
the two cases at the same time. To this end,
we intro

In [4]:
documents = SimpleDirectoryReader(input_files=["../data/2505.23724v1.txt"]).load_data()

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
docs = text_splitter.create_documents([text])

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
faiss_index = FAISS.from_documents(docs, embedding_model)
faiss_index.save_local("faiss_index")

C:\Users\lexil\AppData\Local\Temp\ipykernel_49540\2918464729.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [10]:
llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    model_name="meta-llama/llama-4-scout:free",
    openai_api_key= os.environ["OPENAI_API_KEY"]
)
wrapped_llm = LangChainLLM(llm=llm)

In [17]:
query = "How does SC-LoRA differ from regular LoRA?"
response = llm.invoke([query])
print(response.content)

SC-LoRA stands for scale-aware Low-Rank Adaptation, which is an extension or a variation of the Low-Rank Adaptation (LoRA) method. LoRA is a technique used in machine learning, particularly in the context of large language models and other neural networks, to adapt or fine-tune these models efficiently on specific tasks or datasets.

The primary differences between SC-LoRA and regular LoRA lie in their approach to adapting the model:

1. **Scale Awareness**: SC-LoRA introduces a scale-aware mechanism. This means that SC-LoRA is designed to be aware of and adapt to different scales or magnitudes of the model parameters or the data. This can be particularly useful in scenarios where the input data or the model's parameters have a wide range of values, and a one-size-fits-all adaptation approach might not be optimal.

2. **Dynamic Adaptation**: SC-LoRA might offer a more dynamic adaptation mechanism compared to regular LoRA. While LoRA adapts the model by learning low-rank updates to the 

In [21]:
relevant_docs = faiss_index.similarity_search(query, k=3)

context = " ".join([doc.page_content for doc in relevant_docs])

system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentences maximum and keep the answer concise. "
    f"Context: {context}"
)

messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=query)
]

contextual_response = llm.invoke([query])
print(contextual_response.content)


SC-LoRA and LoRA (Low-Rank Adaptation) are both methods used for efficient fine-tuning of large pre-trained models, particularly in the context of adapting these models to specific tasks or datasets. While they share some similarities, there are key differences between them:

1. **Basic Approach**:
   - **LoRA**: LoRA is a method that adapts a pre-trained model by adding low-rank matrices to its weights. Specifically, for a given layer, it updates the weights by adding a low-rank matrix (often represented as the product of two smaller matrices) to the original weights. This approach allows for efficient adaptation with a relatively small number of additional parameters.
   - **SC-LoRA (Structured Compression and Low-Rank Adaptation)**: SC-LoRA extends the basic LoRA approach by incorporating structured compression. Before applying low-rank adaptation, SC-LoRA first compresses the model by pruning or reducing the dimensionality of the weights in a structured manner. This compression ste

### Part 2: LangChain


In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
faiss_index = FAISS.from_documents(docs, embedding_model)
faiss_index.save_local("faiss_index")
